In [ ]:
# !pip install PyQt5

In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import matplotlib as mpl
%matplotlib qt
import seaborn as sns
import os
from os import path
import datetime
import xlrd
plt.style.use('seaborn-v0_8-ticks')
from scipy.signal import find_peaks
import itertools
from pylab import *
import csv

Downloading data from a database of 15 files

In [ ]:
folder_path_15 = "C:\\Users\\48503\\Desktop\\joint_symbolic\\controlled_breathing"
folders_15 = os.listdir(folder_path_15)
breathing_dict_15 = {folder : {} for folder in folders_15}

for folder_name in folders_15:
    path = os.path.join(folder_path_15, folder_name)
    for file_name in os.listdir(path):
        if file_name.endswith('.csv'):
         df_name = file_name.split('_ODD')[0]
         df = pd.read_csv(os.path.join(path, file_name))
         breathing_dict_15[folder_name][df_name] = df

Downloading data from a database of 56 files

In [3]:
folder_path_56 = "C:\\Users\\48503\\Desktop\\joint_symbolic\\controlled_breathing_full2"
folders_56 = os.listdir(folder_path_56)
breathing_dict_56 = {folder : {} for folder in folders_56}

for folder_name in folders_56:
    folder_path = os.path.join(folder_path_56, folder_name)
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
          df_name = file_name.split('PAC_')[1].split('_')[0] 
          file_path = os.path.join(folder_path, file_name)
          csv_file = open(file_path, 'r')
          try:
              dialect = csv.Sniffer().sniff(csv_file.readline(), ';')   
          except:
              dialect = csv.Sniffer().sniff(csv_file.readline(), ',')   
          csv_file.seek(0)
          df = pd.read_csv(file_path, on_bad_lines='skip', sep=dialect.delimiter)
          if dialect.delimiter == ";":
              for row in df:
                  df[row] = df[row].str.replace(',','.').astype(float)
          breathing_dict_56[folder_name][df_name] = df

Downloading data from a database of 37 files

In [47]:
folder_path_37 = "C:\\Users\\48503\\Desktop\\joint_symbolic\\CONTROL_BREATHING_RECORDINGS"
breathing_dict_keys = ['6_breaths', '10_breaths', '15_breaths', 'baseline']
breathing_dict_37 = {frequency:{} for frequency in breathing_dict_keys}
folders_37 = os.listdir(folder_path_37)

for folder_name in folders_37:
    folder_path = os.path.join(folder_path_37, folder_name)
    df_number = folder_name.split('OCHOTNIK_')[1].split('_')[0] 
    for next_folder_name in os.listdir(folder_path):
        try:
           breathing_frequency = next_folder_name.split('CLEAN_DATA')[1].lower()[1:]
        except:
            breathing_frequency = "RAW_DATA"
        if breathing_frequency in breathing_dict_37:
            next_folder_path = os.path.join(folder_path, next_folder_name)
            for file_name in os.listdir(next_folder_path):
                if file_name.endswith('.csv'): 
                    file_path = os.path.join(next_folder_path, file_name)
                    csv_file = open(file_path, 'r')
                    try:
                        dialect = csv.Sniffer().sniff(csv_file.readline(), ';')   
                    except:
                        dialect = csv.Sniffer().sniff(csv_file.readline(), ',')      
                    csv_file.seek(0)
                    decimal_map = {";" : ",", "," : "."}
                    my_df = pd.read_csv(file_path, encoding='latin-1', decimal=decimal_map[dialect.delimiter], on_bad_lines='skip', sep=dialect.delimiter)
                    breathing_dict_37[breathing_frequency][df_number] = my_df

C:\Users\48503\AppData\Local\Temp\ipykernel_6988\2752246935.py:26: DtypeWarning: Columns (3,13) have mixed types. Specify dtype option on import or set low_memory=False.
  my_df = pd.read_csv(file_path, encoding='latin-1', decimal=decimal_map[dialect.delimiter], on_bad_lines='skip', sep=dialect.delimiter)


One file has errors in rows above index 26966 and mixed columns, to convert object columns to float you need to remove the erroneous lines (they are empty in the preview anyway):

In [ ]:
breathing_dict_37["10_breaths"]["3"].iloc[26966]

In [48]:
breathing_dict_37["10_breaths"]["3"] = breathing_dict_37["10_breaths"]["3"][breathing_dict_37["10_breaths"]["3"].index < 26967] 
breathing_dict_37["10_breaths"]["3"] = breathing_dict_37["10_breaths"]["3"].drop('hbr_s1_d1[umol/L]', axis=1)
obj_cols = breathing_dict_37["10_breaths"]["3"].select_dtypes(include=['object']).apply(lambda col: col.apply(lambda item: float(item.replace(',', '.'))))
breathing_dict_37["10_breaths"]["3"][obj_cols.columns] = obj_cols

In [ ]:
breathing_dict_37["10_breaths"]["3"].tail()

Checking column names in files

In [49]:
def check_column_names(breathing_dict):
    for breath_frequency in breathing_dict:
        for patient_nr, patient_df in breathing_dict[breath_frequency].items(): 
            print(patient_df.columns)                

In [ ]:
check_column_names(breathing_dict_56)

In [50]:
check_column_names(breathing_dict_37)

Index(['DateTime', 'abp_cnap[mmHg]', 'etco2[mmHg]', 'co2[mmHg]', 'rr[rpm]',
       'hbo_s1_d1[umol/L]', 'hbo_s1_d2[umol/L]', 'hbo_s2_d3[umol/L]',
       'hbo_s2_d4[umol/L]', 'hbo_s3_d5[umol/L]', 'hbo_s4_d5[umol/L]',
       'hbo_s4_d6[umol/L]', 'hbo_s5_d6[umol/L]', 'hbr_s1_d1[umol/L]',
       'hbr_s1_d2[umol/L]', 'hbr_s2_d3[umol/L]', 'hbr_s2_d4[umol/L]',
       'hbr_s3_d5[umol/L]', 'hbr_s4_d5[umol/L]', 'hbr_s4_d6[umol/L]',
       'hbr_s5_d6[umol/L]', 'fvl', 'fvr'],
      dtype='object')
Index(['Unnamed: 0', 'time', 'raw_signal', 'cleaned_signal', 'interpolated'], dtype='object')
Index(['DateTime', 'abp_cnap[mmHg]', 'etco2[mmHg]', 'co2[mmHg]', 'rr[rpm]',
       'hbo_s1_d1[umol/L]', 'hbo_s1_d2[umol/L]', 'hbo_s2_d3[umol/L]',
       'hbo_s2_d4[umol/L]', 'hbo_s3_d5[umol/L]', 'hbo_s4_d5[umol/L]',
       'hbo_s4_d6[umol/L]', 'hbo_s5_d6[umol/L]', 'hbr_s1_d1[umol/L]',
       'hbr_s1_d2[umol/L]', 'hbr_s2_d3[umol/L]', 'hbr_s2_d4[umol/L]',
       'hbr_s3_d5[umol/L]', 'hbr_s4_d5[umol/L]', 'hbr_s4_d6

In [40]:
def how_many_have_ecg(breathing_dict):
    for breath_frequency in breathing_dict:
        ecg_number = 0
        patients_with_ecg = []
        for patient_nr, patient_df in breathing_dict[breath_frequency].items(): 
            if "ecg" in patient_df.columns:
                ecg_number += 1
                patients_with_ecg.append(patient_nr)
        print(f"Number of people with ECG for frequency {breath_frequency}: {ecg_number}")  
        print(f"Which people have ecg for frequency {breath_frequency}: {patients_with_ecg} \n")

In [ ]:
how_many_have_ecg(breathing_dict_37)

In [51]:
def unify_column_names(breathing_dict):
    column_names_map = {"fv_r[]":"fv_r", "fv_r[fv_r]":"fv_r", "fvr":"fv_r", "fv_l[]":"fv_l", 
                    "fv_l[fv_l]":"fv_l", "fvl":"fv_l", "ekg[]":"ekg", "ekg[ekg]":"ekg", "ecg":"ekg", "abp_arm[mm_Hg]":"abp_arm", "abp_arm[abp_arm]":"abp_arm", "abp_cnap[mmHg]":"abp_arm", "etco2[mmHg]":"etco2"}
    for breath_frequency in breathing_dict:
        for patient_nr, patient_df in breathing_dict[breath_frequency].items(): 
            patient_df.rename(columns=column_names_map, inplace=True)

In [52]:
unify_column_names(breathing_dict_37)

In [ ]:
check_column_names(breathing_dict_56)

In [53]:
check_column_names(breathing_dict_37)

Index(['DateTime', 'abp_arm', 'etco2', 'co2[mmHg]', 'rr[rpm]',
       'hbo_s1_d1[umol/L]', 'hbo_s1_d2[umol/L]', 'hbo_s2_d3[umol/L]',
       'hbo_s2_d4[umol/L]', 'hbo_s3_d5[umol/L]', 'hbo_s4_d5[umol/L]',
       'hbo_s4_d6[umol/L]', 'hbo_s5_d6[umol/L]', 'hbr_s1_d1[umol/L]',
       'hbr_s1_d2[umol/L]', 'hbr_s2_d3[umol/L]', 'hbr_s2_d4[umol/L]',
       'hbr_s3_d5[umol/L]', 'hbr_s4_d5[umol/L]', 'hbr_s4_d6[umol/L]',
       'hbr_s5_d6[umol/L]', 'fv_l', 'fv_r'],
      dtype='object')
Index(['Unnamed: 0', 'time', 'raw_signal', 'cleaned_signal', 'interpolated'], dtype='object')
Index(['DateTime', 'abp_arm', 'etco2', 'co2[mmHg]', 'rr[rpm]',
       'hbo_s1_d1[umol/L]', 'hbo_s1_d2[umol/L]', 'hbo_s2_d3[umol/L]',
       'hbo_s2_d4[umol/L]', 'hbo_s3_d5[umol/L]', 'hbo_s4_d5[umol/L]',
       'hbo_s4_d6[umol/L]', 'hbo_s5_d6[umol/L]', 'hbr_s1_d1[umol/L]',
       'hbr_s1_d2[umol/L]', 'hbr_s2_d3[umol/L]', 'hbr_s2_d4[umol/L]',
       'hbr_s3_d5[umol/L]', 'hbr_s4_d5[umol/L]', 'hbr_s4_d6[umol/L]',
       'hbr_s

In [54]:
def icmp_dateformat_to_datetime(icmp_time_mark):
    datetime_date = xlrd.xldate_as_datetime(icmp_time_mark, 0)
    datetime_date = datetime_date + datetime.timedelta(hours=1)
    return datetime_date

Checking the percentage of missing values in the ecg, abp, fv and etco2 columns

In [55]:
def print_nan_percent(breathing_dict):
    nan_dict = {breath_frequency : {} for breath_frequency in breathing_dict}
    for breath_frequency in breathing_dict:
        for patient_nr, patient_df in breathing_dict[breath_frequency].items():
            nan_string = ""
            for col_name in ['abp_arm', 'ekg', 'fv_r', 'fv_l', 'etco2']:
                if col_name in patient_df.columns:
                    nan_percent = patient_df[col_name].isnull().sum() * 100 / len(patient_df)
                    if nan_percent != 0.0:
                       nan_string += col_name + " : " + str(round(nan_percent, 2)) + "; "
            nan_dict[breath_frequency][patient_nr] = nan_string
    nan_df = pd.DataFrame(nan_dict)
    return nan_df

In [ ]:
print_nan_percent(breathing_dict_56)

In [56]:
print_nan_percent(breathing_dict_37)

,6_breaths,10_breaths,15_breaths,baseline
1,,,,
10,,NaN,,
12,,,,
13,,,,
14,,fv_r : 0.13; fv_l : 0.13; etco2 : 0.2;,,
15,etco2 : 21.43;,etco2 : 52.85;,,
16,,,,
17,,,,
18,,,,
19,,,,


In [57]:
def delete_keys(to_delete, breathing_dict):
    for breath_freq in breathing_dict:
        for key_to_delete in to_delete:
            breathing_dict[breath_freq].pop(key_to_delete, None)

Removing volunteers number 35 and 10 from base 37

In [62]:
to_delete_37 = ["35", "10"]
delete_keys(to_delete_37, breathing_dict_37)

Adding a datetime column to each dataframe and filling missing values (NaN) with average values

In [59]:
def fill_nan_and_add_time(breathing_dict):    
    for breathing_freq in breathing_dict:
        for patient_number, patient_df in breathing_dict[breathing_freq].items():
            patient_df['DateFormat'] = patient_df['DateTime'].map(icmp_dateformat_to_datetime)
            patient_df.fillna(patient_df.mean(), inplace=True)

In [ ]:
fill_nan_and_add_time(breathing_dict_56)

In [60]:
fill_nan_and_add_time(breathing_dict_37)

In [59]:
print_nan_percent(breathing_dict_37)

,6_breaths,10_breaths,15_breaths,baseline
1,,,,
12,,,,
13,,,,
14,,,,
15,,,,
16,,,,
17,,,,
18,,,,
19,,,,
2,,,,


Function finding peaks, diff is RR for ECG, but you need to multiply it by 0.005 (1/(200 Hz)), peaks are sample numbers corresponding to peaks, signal[peaks] are signal values corresponding to these sample numbers

In [61]:
def get_peaks(signal):
    peaks, _ = find_peaks(signal, distance=115)
    difference = np.diff(peaks)  
    return peaks, signal[peaks], difference

Plotting the first 600 samples of each signal and marking the found peaks ("x")

In [22]:
def check_peak_detection_accuracy(plot_folder_path, data_dict):
    for breath_frequency in data_dict:
        for dataframe_nr, dataframe in data_dict[breath_frequency].items():       
            fig, ax = plt.subplots(5, 1, tight_layout = True, figsize=(16, 32)) 
            for i, col_name in enumerate(['abp_arm', 'ekg', 'fv_l', 'fv_r', 'etco2']):
                if col_name in dataframe.columns:
                    x = dataframe[col_name].iloc[:600]
                    ax[i].plot(x)
                    my_peaks = get_peaks(x)
                    ax[i].plot(my_peaks[0], my_peaks[1], "x")
                    ax[i].set_ylabel(f'{col_name}', fontsize=25)
                    ax[i].tick_params(axis='both', labelsize=18)
                    ax[i].grid(True)
            ax[3].set_xlabel("sample number", fontsize = 28)
            savefig_path = plot_folder_path+f"\\{breath_frequency}\\{dataframe_nr}_plot.pdf"
            plt.savefig(savefig_path, bbox_inches='tight', dpi=600)
            plt.show()
            plt.close()

In [15]:
folder_path = "C:\\Users\\48503\\Desktop\\joint_symbolic\\timeseries_plots\\56"
check_peak_detection_accuracy(folder_path, breathing_dict_56)

In [23]:
folder_path = "C:\\Users\\48503\\Desktop\\joint_symbolic\\timeseries_plots\\37"
check_peak_detection_accuracy(folder_path, breathing_dict_37)

Removal of previously excluded patients number 2, 6 and 33 and additionally 14 and 31 (bad signal quality) from the base 56

In [89]:
to_delete_56 = ["2", "6", "33", "14", "31"]
delete_keys(to_delete_56, breathing_dict_56)

In [36]:
def make_empty_matrix():
    my_permutations = [''.join(map(str, triplet)) for triplet in list((itertools.product([1, 0], repeat=3)))]
    my_matrix = pd.DataFrame()
    for triplet_1 in my_permutations: 
       for triplet_2 in my_permutations:
            my_matrix.at["'"+triplet_1, "'"+triplet_2] = 0 #dash needed so that R does not remove the initial zeros 
    return my_matrix

In [37]:
def get_diff_df(dataframe, is_fv, is_left, is_ekg):
    sap = get_peaks(dataframe['abp_arm'])[1]
    if is_fv:
        sap = get_peaks(dataframe['fv_l'])[1]
        if not is_left:
            sap = get_peaks(dataframe['fv_r'])[1]

    rr = get_peaks(dataframe['abp_arm'])[2]*0.005 #in those without an ECG signal, we use ABP
    if is_ekg:
        rr = get_peaks(dataframe['ekg'])[2]*0.005
        
    sap_diff = np.diff(sap)
    rr_diff = np.diff(rr)
    diff_dataframe = pd.DataFrame(list(zip(sap_diff, rr_diff)), columns=['sap_diff', 'rr_diff'])
    return diff_dataframe

In [38]:
def make_and_save_heatmap(relative_array, breath_frequency, is_fv, heatmap_path):
    my_custom_palette = sns.color_palette("ch:-.24", as_cmap=True)
    my_heatmap = sns.heatmap(relative_array, cmap = my_custom_palette, annot=True)
    my_heatmap.xaxis.tick_top()
    my_heatmap.xaxis.set_ticks_position('none')
    fontsize = 10
    plt.xlabel('SAP', fontsize = fontsize) 
    my_heatmap.xaxis.set_label_position('top') 
    plt.ylabel('RR', fontsize = fontsize) 
    if is_fv:
        plt.savefig(heatmap_path+f"\\{breath_frequency}_heatmap_fv.pdf", bbox_inches='tight', dpi=600)
        relative_array.to_csv(heatmap_path+f"\\{breath_frequency}_matrix_fv.csv")
    else:
        plt.savefig(heatmap_path+f"\\{breath_frequency}_heatmap_abp.pdf", bbox_inches='tight', dpi=600)
        relative_array.to_csv(heatmap_path+f"\\{breath_frequency}_matrix_abp.csv")
    plt.show()
    plt.close()

In [39]:
def save_jsd(is_fv, jsd_df, jsd_path):
    if is_fv:
        jsd_df.T.to_csv(jsd_path+"\\jsd_fv.csv")
    else:    
        jsd_df.T.to_csv(jsd_path+"\\jsd_abp.csv")

In [40]:
def add_mean_params(jsd_df, dataframe, is_left, patient_number, freq):
    window_size = 2000
    for col_name in ['abp_arm', 'etco2']:
        if col_name in dataframe.columns:
            nan_percent = dataframe[col_name].isnull().sum() * 100 / len(dataframe)
            if nan_percent != 100.0:
                jsd_df.at["mean_"+col_name+"_"+freq, patient_number] = dataframe[col_name].rolling(window=window_size).mean().dropna().mean()
    if is_left:
        jsd_df.at["mean_fvr/l_"+freq, patient_number] = dataframe['fv_l'].rolling(window=window_size).mean().dropna().mean()
    else:
        jsd_df.at["mean_fvr/l_"+freq, patient_number] = dataframe['fv_r'].rolling(window=window_size).mean().dropna().mean()
    return jsd_df

In [41]:
def joint_symbolic(data_dict, is_fv, metadata_path, heatmap_path, jsd_path, is_ekg):
    jsd_df = pd.DataFrame()
    metadata = pd.read_csv(metadata_path, sep = ";")
    for freq in data_dict:
        freq_matrix = make_empty_matrix()
        for patient_number, dataframe in data_dict[freq].items():
            is_left = bool(metadata.at[metadata.loc[metadata['L.poj.'] == int(patient_number)].index[0], 'FV_LEWA_MCA'])
            patient_freq_matrix = make_empty_matrix()
            diff_df = get_diff_df(dataframe, is_fv, is_left, is_ekg)
            
            for col_name in ['sap_diff', 'rr_diff']:
                diff_df[f'{col_name}_binary'] = diff_df.apply(lambda df_row: 1 if df_row[col_name] > 0 else 0, axis = 1)
            
            for index in range(0, len(diff_df)):
                sap_triplet =  ''.join(diff_df['sap_diff_binary'].iloc[index:index+3].astype(str))
                rr_triplet =  ''.join(diff_df['rr_diff_binary'].iloc[index:index+3].astype(str))
                if len(rr_triplet) > 2 and len(sap_triplet) > 2:
                     freq_matrix.at["'"+rr_triplet, "'"+sap_triplet] += 1 #dash needed so that R does not remove the initial zeros 
                     patient_freq_matrix.at["'"+rr_triplet, "'"+sap_triplet] += 1 
            
            relative_patient_matrix = patient_freq_matrix.div(np.sum(patient_freq_matrix.values))   
            signal_map = {True : 'FV_', False : 'ABP_'}
            jsd_df.at[signal_map[is_fv]+"jsd_sym_"+freq, patient_number] = np.sum(np.diag(relative_patient_matrix)) 
            jsd_df.at[signal_map[is_fv]+"jsd_diam_"+freq, patient_number] = np.sum(np.diag(np.rot90(relative_patient_matrix)))
            jsd_df = add_mean_params(jsd_df, dataframe, is_left, patient_number, freq)
            
        relative_freq_matrix = freq_matrix.div(np.sum(freq_matrix.values))        
        make_and_save_heatmap(relative_freq_matrix, freq, is_fv, heatmap_path)
    save_jsd(is_fv, jsd_df, jsd_path)     

In [42]:
jsa_path = "C:\\Users\\48503\\Desktop\\joint_symbolic\\"

Joint symbolic for base 56

In [120]:
meta_path_56 = jsa_path + "dane_sonata\\dane_sonata_56.csv"
heatmap_path_56 = jsa_path + "heatmaps\\56"
jsd_path_56 = jsa_path + "jsd\\56"

In [98]:
joint_symbolic(breathing_dict_56, False, meta_path_56, heatmap_path_56, jsd_path_56, True) #abp
joint_symbolic(breathing_dict_56, True, meta_path_56, heatmap_path_56, jsd_path_56, True)  #fv

Joint symbolic for base 37

In [43]:
meta_path_37 = jsa_path + "dane_sonata\\dane_sonata_37.csv"
heatmap_path_37 = jsa_path + "heatmaps\\37"
jsd_path_37 = jsa_path + "jsd\\37"

In [44]:
joint_symbolic(breathing_dict_37, False, meta_path_37, heatmap_path_37, jsd_path_37, False) #abp
joint_symbolic(breathing_dict_37, True, meta_path_37, heatmap_path_37, jsd_path_37, False)  #fv

Merging files with jsd and average parameters calculated for fv and abp

In [45]:
jsd_path_37 = jsa_path + "jsd\\37\\"
jsd_abp = pd.read_csv(jsd_path_37+'jsd_abp.csv', index_col='Unnamed: 0')
jsd_fv = pd.read_csv(jsd_path_37+'jsd_fv.csv', index_col='Unnamed: 0')

jsd_fv = jsd_fv.drop(columns=jsd_abp.columns.intersection(jsd_fv.columns)) #removing repeated columns

jsd = jsd_abp.merge(jsd_fv, left_index=True, right_index=True, how='inner')
jsd_sorted = jsd.sort_index()
jsd_sorted.to_csv(jsd_path_37+'jsd_all.csv', index_label='patient_id')

Changing the form of the relative frequency matrix to a simple table to draw 3D histograms in R

In [142]:
def transform_matrix(heatmap_path):
    save_path = heatmap_path + "\\transformed\\"
    for my_file_name in os.listdir(heatmap_path):
        if my_file_name.endswith('.csv'):
           dataframe = pd.read_csv(heatmap_path + f"\\{my_file_name}", index_col='Unnamed: 0')
           new_df = pd.DataFrame(columns=['RR', 'SAP', 'relative frequency'])

           for idx in dataframe.index:
              for col in dataframe.columns:
                 new_df = pd.concat([new_df, pd.DataFrame({'RR': [idx], 'SAP': [col], 'relative frequency': [dataframe.at[idx, col]]})], ignore_index=True)
           new_df.to_csv(save_path + f"\\{my_file_name}", index=False)

In [143]:
transform_matrix(heatmap_path_37)

C:\Users\48503\AppData\Local\Temp\ipykernel_21888\4038159935.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, pd.DataFrame({'RR': [idx], 'SAP': [col], 'relative frequency': [dataframe.at[idx, col]]})], ignore_index=True)
C:\Users\48503\AppData\Local\Temp\ipykernel_21888\4038159935.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, pd.DataFrame({'RR': [idx], 'SAP': [col], 'relative frequency': [dataframe.at[idx, col]]})], ignore_index=